In [1]:
df = spark.read.format("delta").load("abfss://WSP_MartinicaRealState@onelake.dfs.fabric.microsoft.com/LK_brz_MartinicaRealState.Lakehouse/Tables/leads")
df.createOrReplaceTempView("tmp_lea")

df = spark.read.format("delta").load("abfss://WSP_MartinicaRealState@onelake.dfs.fabric.microsoft.com/LK_brz_MartinicaRealState.Lakehouse/Tables/sales")
df.createOrReplaceTempView("tmp_sal")

StatementMeta(, e537f40d-9271-408a-bd4e-4cc80f633278, 3, Finished, Available, Finished)

In [12]:
%%sql
MERGE INTO leads AS TG
USING tmp_lea AS OS
ON TG.LeadID = OS.LeadID
   AND TG.LeadDate = OS.LeadDate

WHEN MATCHED THEN
    UPDATE SET
        TG.ClientID     = OS.ClientID,
        TG.PropertyID   = OS.PropertyID,
        TG.CampaignID   = OS.CampaignID,
        TG.LeadSource   = OS.LeadSource

WHEN NOT MATCHED THEN
    INSERT (LeadID, ClientID, PropertyID, CampaignID, LeadDate, LeadSource)
    VALUES (OS.LeadID, OS.ClientID, OS.PropertyID, OS.CampaignID, OS.LeadDate, OS.LeadSource);


StatementMeta(, e537f40d-9271-408a-bd4e-4cc80f633278, 14, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [16]:
%%sql
MERGE INTO sales AS target
USING tmp_sal AS source
ON target.SaleID = source.SaleID
   AND target.SaleDate = source.SaleDate
WHEN MATCHED THEN
    UPDATE SET 
        target.PropertyID   = source.PropertyID,
        target.ClientID     = source.ClientID,
        target.BrokerID     = source.BrokerID,
        target.SalePriceUSD = source.SalePriceUSD
WHEN NOT MATCHED BY TARGET THEN
    INSERT (SaleID, PropertyID, ClientID, BrokerID, SaleDate, SalePriceUSD)
    VALUES (source.SaleID, source.PropertyID, source.ClientID, source.BrokerID, source.SaleDate, source.SalePriceUSD);


StatementMeta(, e537f40d-9271-408a-bd4e-4cc80f633278, 18, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>